In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
DATA_START = pd.to_datetime('2015-01-01')
DATASET_END = pd.to_datetime('2018-01-01')
NUM_SAMPLES = int(1e5)
MIN_APT_SIZE = 600

In [3]:
listing_dates = DATA_START + pd.to_timedelta(np.random.randint(0, 365*3, size=NUM_SAMPLES), unit='D')

In [4]:
house_qualities = np.random.exponential(size=NUM_SAMPLES)
years_since_remodel = np.random.exponential(scale=1e1,size=NUM_SAMPLES)
size_bonus = np.random.rand(NUM_SAMPLES) * 100
listing_price = np.clip(house_qualities * 400000 - years_since_remodel * 1000, 1000, None)

In [5]:
sqft = house_qualities * 1000 + size_bonus + MIN_APT_SIZE

In [6]:
bedroom_bonus = np.random.rand(NUM_SAMPLES) * 1000
bedrooms = np.floor(np.power((sqft + bedroom_bonus) * house_qualities // 300, .4))
#If more bedrooms per same space, fewer bathrooms can fit (hypothetical to make data more interesting)
bathroom_bonus = np.random.rand(NUM_SAMPLES) * 1000 * house_qualities - 1000 * bedrooms
bathrooms = np.floor(np.power(np.clip((sqft + bathroom_bonus) // 10, 0, None) + 1, .3))

In [7]:
#inspection

In [8]:
print('bedroom listing_price correlation,', np.corrcoef(bedrooms, listing_price)[0,1])

bedroom listing_price correlation, 0.9601735751001105


In [9]:
print('bathrooms listing_price correlation,', np.corrcoef(bathrooms, listing_price)[0,1])

bathrooms listing_price correlation, 0.3057127272654651


In [10]:
print('bedroom bathroom correlation,', np.corrcoef(bedrooms, bathrooms)[0,1])
print('bedroom min mean max', bedrooms.min(), bedrooms.mean(), bedrooms.max())
print('bathroom miin mean max',bathrooms.min(), bathrooms.mean(), bathrooms.max())

bedroom bathroom correlation, 0.12370833415798833
bedroom min mean max 0.0 1.51545 12.0
bathroom miin mean max 1.0 2.6779 8.0


In [11]:
(listing_dates - DATA_START).days.values.astype(float)

array([ 581.,   96.,  472., ...,  295., 1087.,  465.])

In [12]:
time_bonus = (listing_dates - DATA_START).days.values.astype(float)

In [13]:
bath_bed_penalty = bathrooms - bedrooms
bath_bed_ind = bathrooms > bedrooms

In [14]:
broker = np.random.randint(0, 2, NUM_SAMPLES)

In [15]:
sell_inv = bedrooms * bathrooms/bedrooms.std()/bathrooms.std() + bedrooms/bedrooms.std()
sell_inv += broker
sell_inv += time_bonus/time_bonus.std()
sell_inv += bathrooms/bathrooms.std() + sqft/sqft.std()
sell_inv += house_qualities*2
sell_inv += size_bonus/size_bonus.std()
sell_inv -= years_since_remodel/years_since_remodel.std()
sell_inv += np.random.rand(NUM_SAMPLES) + 10
sell_inv[bath_bed_ind] = sell_inv[bath_bed_ind] - bath_bed_penalty[bath_bed_ind]/2
sales_duration = np.floor(1/sell_inv * 3000).astype(int)

In [16]:
sales_duration.min(), sales_duration.mean(), sales_duration.std(), sales_duration.max()

(22, 139.38828, 40.206077881852636, 466)

In [17]:
sales_duration

array([141, 222, 152, ..., 184, 159, 155])

In [18]:
def plt_clipped(a):
    clip_max = np.percentile(a, 99)
    a = np.clip(a, a.min(), clip_max)
    #a = a[a < clip_max]
    plt.hist(a, bins=100)

In [19]:
sales_date = listing_dates + pd.to_timedelta(sales_duration,unit='D')

In [20]:
final_df = pd.DataFrame({'SalesDate':sales_date, 'ListingDate':listing_dates,
              'bedrooms':bedrooms,'bathrooms':bathrooms,'sqft':sqft,'years_since_remodel':years_since_remodel,
                        'broker': broker})

In [21]:
final_df.loc[final_df.SalesDate > DATASET_END, 'SalesDate'] = np.nan

In [22]:
cnt = 0
for ridx in np.random.choice(final_df.shape[0], final_df.shape[0]//6, replace = False):
    cnt += 1
    if cnt % 10000 == 0:
        print(cnt, cnt/final_df.shape[0]*6)
    if final_df.loc[ridx,'broker']:
        if np.random.rand() > .5:
            final_df.loc[ridx, 'sqft'] += 100
            final_df.loc[ridx, 'bedrooms'] = np.nan
        else:
            final_df.loc[ridx, 'bedrooms'] += 1
            final_df.loc[ridx, 'sqft'] = np.nan
    else:
        feature = np.random.randint(2, final_df.shape[1])
        final_df.iloc[ridx, feature] = np.nan
    final_df.loc[ridx,'broker'] = np.nan


10000 0.6000000000000001


In [23]:
final_df.to_csv('houselistings_simulated.csv')

In [24]:
final_df.isnull().sum()

ListingDate                0
SalesDate              11894
bathrooms               1581
bedrooms                5844
broker                 16666
sqft                    5971
years_since_remodel     1668
dtype: int64

In [25]:
final_df

,ListingDate,SalesDate,bathrooms,bedrooms,broker,sqft,years_since_remodel
0,2016-08-04,2016-12-23,3.0,2.0,NaN,NaN,0.394278
1,2015-04-07,2015-11-15,3.0,0.0,0.0,719.022093,7.305659
2,2016-04-17,2016-09-16,2.0,NaN,NaN,1265.584991,17.550252
3,2016-01-05,2016-05-19,1.0,3.0,0.0,2827.240245,29.689586
4,2016-04-28,2016-11-28,3.0,0.0,0.0,686.116778,4.670000
5,2016-07-22,2016-12-13,3.0,1.0,1.0,1207.184287,6.342823
6,2016-02-23,2016-06-27,1.0,3.0,0.0,2463.018617,15.050084
7,2017-03-29,2017-07-31,3.0,1.0,1.0,1368.532837,0.040317
8,2015-06-02,2015-10-22,1.0,2.0,0.0,1988.535751,12.507711
9,2017-03-13,2017-07-21,1.0,2.0,0.0,1718.652239,5.658188
